# Modelling 

Within this jupyter notebook - we will be creating our predictive model for the survivors of the titanic shipwreck. 

Evaluation metric being Accuracy for our classification models. 

After preparing the data we will then create the following models: 
- Logistic Regression 
- Random Forest Classifier 
- XGBoost Classifier 

In [1]:
# setting up the packages and modules that are needed for analysis
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
# Models from scikit learn 

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations 

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV

from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_curve

%matplotlib inline

In [2]:
# read the data in to the notebook - we will have a look at the training set first 
training_df = pd.read_csv('../Data/train.csv')

In [3]:
training_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
training_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# transform for the missing values: numeric first: 
training_df['Age'] = training_df['Age'].fillna(training_df.groupby(['Sex', 'Pclass'])['Age'].transform('mean').round(1))

In [6]:
training_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

For the Cabin feature now: 

In [7]:
training_df['Cabin'].head(10)
# we can take the first letter to signify which cabin they sat in

0     NaN
1     C85
2     NaN
3    C123
4     NaN
5     NaN
6     E46
7     NaN
8     NaN
9     NaN
Name: Cabin, dtype: object

In [8]:
#creates categories based on the cabin letter (n stands for null)
#in this case we will treat null values like it's own category

training_df['cabin_adv'] = training_df.Cabin.apply(lambda x: str(x)[0])
training_df['cabin_adv'] = training_df['cabin_adv'].str.lower()

In [9]:
training_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cabin_adv
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,n
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,c
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,n
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,c
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,n
5,6,0,3,"Moran, Mr. James",male,26.5,0,0,330877,8.4583,NaN,Q,n
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,e
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,n
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,n
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,n


In [10]:
training_df['cabin_adv'].value_counts()

n    687
c     59
b     47
d     33
e     32
a     15
f     13
g      4
t      1
Name: cabin_adv, dtype: int64

In [11]:
#comparing surivial rate by cabin
pd.pivot_table(training_df,index='Survived',columns='cabin_adv', values = 'Name', aggfunc='count')

cabin_adv,a,b,c,d,e,f,g,n,t
Survived,,,,,,,,,
0,8.0,12.0,24.0,8.0,8.0,5.0,2.0,481.0,1.0
1,7.0,35.0,35.0,25.0,24.0,8.0,2.0,206.0,NaN


In [14]:
# we can now drop the cabin feature 

training_df.drop(['Cabin'],axis=1, inplace = True)

In [15]:
training_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,cabin_adv
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,n
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,c
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,n
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,c
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,n
